In [0]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *


#Read input csv file

path = "/FileStore/tables/electronic_card_transactions_may_2021_csv_tables.csv"
df2 = spark.read.format("csv").option("header","true").option('inferschema','true')\
  .option("delimiter", ",").option("quote", "\"").option("escape", "\"")\
    .option("multiline", "true").load(path)

df_trimble.write.option("header","true").mode("overwrite").option("inferSchema","true").option("delimiter", ",").option("quote", "\"").option("escape", "\"")\
      .option("multiline", "true").parquet("/mnt/curated/trimble/"+yesterday_name)

#Copying one to another dataframe
transactions = df2

#Example for lazy evolution
transactions.show()

dataframe.select('STATUS').distinct().show()

#Selecting specific columns
tnx1 = transactions.select('Series_reference','Period','Data_Value','STATUS')
tnx1.count()

#Data filtering 
l = ['C','F']
tnx2 = tnx1.filter(tnx1.STATUS.isin(l))
tnx2.count()

#windowing function for aggregation
w = Window().partitionBy("Series_reference","Period").orderBy(col("Period").desc())
tnx3 = tnx2.withColumn("max_Data_value", max('Data_Value').over(w))
txn4 = tnx3.select('Series_reference','Period','max_Data_Value','STATUS')


threshold = sc.parallelize([("2001.03", "2000"), ("2002.03", "2000"),
("2003.03", "2000"), ("2004.03", "2000"),
("2005.03", "4000"), ("2005.03", "5000"),
("2007.03", "4000"), ("2008.03", "5000"),
("2009.03", "4000"), ("2010.03", "5000"),
("2011.03", "6000"), ("2012.03", "12000"),
("2013.03", "6000"), ("2014.03", "12000"),
("2015.03", "6000"), ("2016.03", "12000"),
("2017.03", "8000"), ("2018.03", "9000"),
("2019.03", "8000"), ("2002.03", "9000"),
("2021.03", "8000")])

#create RDD, convert RDD to DF
threshold = spark.createDataFrame(threshold).toDF("tsh_Period", "tsh_Value")
threshold.display()

#Join 2 dataframes
sdf_txn_tsh = txn4.alias('txn')\
                                .join(threshold.alias('threshold'),(col('txn.Period') == col('threshold.tsh_Period')), how="left")\
.select('Series_reference','Period','max_Data_Value','STATUS','tsh_Value')

#isNotnull, withcolumn, When, withcolumnRenamed, type casting


# sdf_txn_tsh.show()
sdf_txn_tsh = sdf_txn_tsh.filter(sdf_txn_tsh.max_Data_Value.isNotNull())
sdf_txn_tsh.show()


# sdf_txn_tsh_1 = sdf_txn_tsh.fillna( { 'tsh_Value':0})
# sdf_txn_tsh_1.show()
sdf_txn_tsh_2 = sdf_txn_tsh_1.withColumn('Is_Profit',when(sdf_txn_tsh_1.max_Data_Value > sdf_txn_tsh_1.tsh_Value , 'Y').otherwise('N').cast(StringType()))


sdf_txn_tsh_2 = sdf_txn_tsh_2.withColumnRenamed('Is_Profit','Profit?')

sdf_txn_tsh_2.show()

#Duplicate check
sdf_txn_tsh_2 = sdf_txn_tsh_2.withColumn("Dup", count("*").over(Window.partitionBy('Series_reference','Period')))

# Keep the count
Duplicates_count = sdf_txn_tsh_2.filter(sdf_txn_tsh_2.Dup > 1).count()

#Removing duplicates - usage of row number

w = Window().partitionBy('Series_reference','Period').orderBy(col("max_Data_Value").desc())
sdf_txn_tsh_2 = sdf_txn_tsh_2.withColumn("rn", row_number().over(w)).where(col("rn") == 1).select("*")
re
#Write back the df to any repository

sdf_parq_file  = spark.read.format('parquet').option("header","true")\
  .option("delimiter", ",").option("quote", "\"").option("escape", "\"")\
    .option("multiline", "true").load('/FileStore/tables/electronic_card_transactions_may_2021_tables')

sdf_parq_file.show()


In [0]:
union of 3 dataframes - reduce